References:

* https://astroquery.readthedocs.io/en/stable/jplhorizons/jplhorizons.html
* https://docs.poliastro.space/en/stable/examples/Visualizing%20the%20SpaceX%20Tesla%20Roadster%20trip%20to%20Mars.html
* https://docs.poliastro.space/en/stable/examples/Analyzing%20NEOs.html
* https://docs.poliastro.space/en/stable/examples/Analyzing%20the%20Parker%20Solar%20Probe%20flybys.html

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import time

import astropy.units as u
from astropy.coordinates import CartesianRepresentation
from astropy.time import Time
from astropy.table import Table
from astroquery.jplhorizons import Horizons

from poliastro.bodies import Sun, Earth
from poliastro.frames import Planes
from poliastro.iod import lambert
from poliastro.plotting import OrbitPlotter3D
from poliastro.ephem import Ephem
from poliastro.twobody import Orbit

# Code Configuration

In [ ]:
# name = "C/2013 A1"
# epoch = Time("2014-10-25")  # last perihelion
# name = "C/2024 E1"
# epoch = Time("2026-01-21")  # perihelion
name = "Loverboy"
epoch = Time("2033-06-15T12:00:00")  # perihelion

custom_comet = True  # toggle code for custom comet
custom_orbital_elements = {
    "a": 10474.06 * u.au,         # Semi-major axis
    "ecc": 0.9998956528860143 * u.one,     # Eccentricity
    "inc": 100.88290133343503 * u.deg,      # Inclination
    "raan": 232.4318366430011 * u.deg,    # Right ascension of ascending node
    "argp": 335.5337773673682 * u.deg,    # Argument of periapsis
    "nu": 0 * u.deg,        # True anomaly
}

In [ ]:
# Generate state vectors of target body
if not custom_comet:  # query target body from JPL Horizons database
    start = epoch - 200 * u.day
    stop = epoch + 200 * u.day

    obj = Horizons(
        id=name, location="@10", epochs={"start": start.iso, "stop": stop.iso, "step": "5d"}
    )

    vec = obj.vectors(refplane="ecliptic")

else:  # generate custom comet
    orbit = Orbit.from_classical(Sun, **custom_orbital_elements, epoch=epoch)  # create orbit using Poliastro

    times = epoch + np.arange(-200, 200, 5) * u.day  # same 5 day time step as above
    states = [orbit.propagate((t - epoch).to(u.s)) for t in times]

    # Convert state vectors into arrays with units
    r_vectors = np.array([s.r.to_value(u.km) for s in states]) * u.km
    v_vectors = np.array([s.v.to_value(u.km / u.s) for s in states]) * u.km / u.s

    # Construct an Astropy Table with units
    vec = Table(
        {
            "datetime_jd": times.jd * u.day,
            "x": r_vectors[:, 0],
            "y": r_vectors[:, 1],
            "z": r_vectors[:, 2],
            "vx": v_vectors[:, 0],
            "vy": v_vectors[:, 1],
            "vz": v_vectors[:, 2],
        }
    )

In [ ]:
# 3D interactive orbit viewer
frame = OrbitPlotter3D(plane=Planes.EARTH_ECLIPTIC)
frame.set_attractor(Sun)

orbit = CartesianRepresentation(
    vec["x"],
    vec["y"],
    vec["z"],
)
frame.plot_trajectory(orbit, label=name)

# Lambert

In [ ]:
tofs = np.linspace(100, 1500, 200)
epochs = Time(vec["datetime_jd"], format="jd", scale="tdb")

results = []

start = time.time()
for i, t_comet in enumerate(epochs):
    r_comet = u.Quantity(list(vec[i]["x", "y", "z"]), unit=vec["x"].unit)
    v_comet = u.Quantity(list(vec[i]["vx", "vy", "vz"]), unit=vec["vx"].unit)

    res = []

    for tof in tofs:
        tof = tof * u.day
        t_earth = t_comet - tof
        earth_pos = Ephem.from_body(Earth, t_earth, attractor=Sun, plane=Planes.EARTH_ECLIPTIC)
        r_earth, v_earth = earth_pos.rv(t_earth)

        v_i_1, v_f_1 = lambert(Sun.k, r_earth, r_comet, tof, lowpath=True)
        v_i_2, v_f_2 = lambert(Sun.k, r_earth, r_comet, tof, lowpath=False)

        v_inf_e_1 = np.linalg.norm(v_i_1 - v_earth)
        v_inf_c_1 = np.linalg.norm(v_f_1 - v_comet)

        v_inf_e_2 = np.linalg.norm(v_i_2 - v_earth)
        v_inf_c_2 = np.linalg.norm(v_f_2 - v_comet)

        if v_inf_e_1 > v_inf_e_2:
            v_inf_arr = v_inf_c_2
            c3 = np.linalg.norm(v_inf_e_2)**2
        else:
            v_inf_arr = v_inf_c_1
            c3 = np.linalg.norm(v_inf_e_1)**2

        res.append([c3.to_value(u.km**2/u.s**2), v_inf_arr.to_value(u.km/u.s)])

    results.append(res)
stop = time.time()

results = np.array(results)

print(f"{(len(epochs) * len(tofs))/(stop - start)} iterations/sec")

In [ ]:
c3 = results[:, :, 0]
v_inf = results[:, :, 1]

c3[c3 > 200] = np.nan
v_inf[v_inf > 60] = np.nan

In [ ]:
plt.figure(figsize=(12, 8), dpi=300)

t = [e.mjd for e in epochs]

CS = plt.contour(t, tofs, c3.T, levels=10, colors="blue", linewidths=0.5)
plt.clabel(CS, CS.levels, inline=True)

CS = plt.contour(t, tofs, v_inf.T, levels=10, colors="red", linewidths=0.5)
plt.clabel(CS, CS.levels, inline=True)

# Dummy points for labels
plt.plot(t[0], tofs[0], lw=0.5, color="b", label="C3 (km2/s2)")
plt.plot(t[0], tofs[0], lw=0.5, color="r", label="v_inf (km/s)")

plt.legend(loc=4)
plt.title(name)
plt.xlabel("Arrival Date (MJD)")
plt.ylabel("TOF (days)")

In [ ]:
idx = np.unravel_index(np.nanargmin(c3), c3.shape)
best = results[idx]

print(f"Best trajectory: arrival {epochs[idx[0]].mjd} MJD C3={best[0]} km2/s2, v_inf={best[1]} km/s, tof={tofs[idx[1]]} days")